### Neural Network Model for Home Credit Complete Dataset

The following notebooks aims to train a Neural Network on the dataset of home credit which already have a lot of new features created througha feature engeneering process

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV 
import tensorflow as tf
from tensorflow import keras
import warnings
warnings.filterwarnings('ignore')
plt.style.use('fivethirtyeight')

In [60]:
df = pd.read_csv('app_train_cleaned.csv')

df

,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,EXT_SOURCE_2,AMT_GOODS_PRICE,AMT_ANNUITY,CNT_FAM_MEMBERS,DAYS_LAST_PHONE_CHANGE,CLIENT_AGE,HOUR_APPR_PROCESS_START,WEEKDAY_APPR_PROCESS_START,DAYS_ID_PUBLISH,DAYS_REGISTRATION,DAYS_BIRTH,REGION_POPULATION_RELATIVE,NAME_FAMILY_STATUS,AMT_CREDIT,AMT_INCOME_TOTAL,ORGANIZATION_TYPE,SK_ID_CURR,TARGET
0,2.0,2.0,0.262949,351000.0,24700.5,1.0,-1134.0,25.920548,10,WEDNESDAY,-2120,-3648.0,-9461,0.018801,Single / not married,406597.5,202500.0,Business Entity Type 3,100002,1
1,1.0,1.0,0.622246,1129500.0,35698.5,2.0,-828.0,45.931507,11,MONDAY,-291,-1186.0,-16765,0.003541,Married,1293502.5,270000.0,School,100003,0
2,0.0,0.0,0.555912,135000.0,6750.0,1.0,-815.0,52.180822,9,MONDAY,-2531,-4260.0,-19046,0.010032,Single / not married,135000.0,67500.0,Government,100004,0
3,2.0,2.0,0.650442,297000.0,29686.5,2.0,-617.0,52.068493,17,WEDNESDAY,-2437,-9833.0,-19005,0.008019,Civil marriage,312682.5,135000.0,Business Entity Type 3,100006,0
4,0.0,0.0,0.322738,513000.0,21865.5,1.0,-1106.0,54.608219,11,THURSDAY,-3458,-4311.0,-19932,0.028663,Single / not married,513000.0,121500.0,Religion,100007,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,0.0,0.0,0.681632,225000.0,27558.0,1.0,-273.0,25.553425,15,THURSDAY,-1982,-8456.0,-9327,0.032561,Separated,254700.0,157500.0,Services,456251,0
307507,0.0,0.0,0.115992,225000.0,12001.5,1.0,0.0,56.917808,8,MONDAY,-4090,-4388.0,-20775,0.025164,Widow,269550.0,72000.0,XNA,456252,0
307508,6.0,6.0,0.535722,585000.0,29979.0,1.0,-1909.0,41.002740,9,THURSDAY,-5150,-6737.0,-14966,0.005002,Separated,677664.0,153000.0,School,456253,0
307509,0.0,0.0,0.514163,319500.0,20205.0,2.0,-322.0,32.769863,9,WEDNESDAY,-931,-2562.0,-11961,0.005313,Married,370107.0,171000.0,Business Entity Type 1,456254,1


In [48]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

for i in df.select_dtypes(include='object').columns:
    enc = OrdinalEncoder()
    #print(train_df[i])
    df[i].fillna(df[i].mode()[0], inplace=True)
    df[i]= enc.fit_transform(df[[i]])


#Standerization
scaler = StandardScaler()
app_train_scaled = scaler.fit_transform(df.drop(columns='TARGET'))
app_train_df = pd.DataFrame(app_train_scaled, index=df.index, columns=df.drop(columns='TARGET').columns)
app_train_df['TARGET'] = df['TARGET']
app_train_df

,OBS_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,EXT_SOURCE_2,AMT_GOODS_PRICE,AMT_ANNUITY,CNT_FAM_MEMBERS,DAYS_LAST_PHONE_CHANGE,CLIENT_AGE,HOUR_APPR_PROCESS_START,WEEKDAY_APPR_PROCESS_START,DAYS_ID_PUBLISH,DAYS_REGISTRATION,DAYS_BIRTH,REGION_POPULATION_RELATIVE,NAME_FAMILY_STATUS,AMT_CREDIT,AMT_INCOME_TOTAL,ORGANIZATION_TYPE,SK_ID_CURR,TARGET
0,0.240632,0.249898,-1.316049,-0.507236,-0.166146,-1.265719,-0.206990,-1.506880,-0.631821,1.353458,0.579154,0.379837,1.506880,-0.149452,1.306965,-0.478095,0.142129,-1.194830,-1.733423,1
1,-0.175863,-0.170305,0.564499,1.599974,0.592666,-0.167639,0.163108,0.166821,-0.325620,-0.972656,1.790855,1.078697,-0.166821,-1.252750,-0.404071,1.725450,0.426792,0.466663,-1.733413,0
2,-0.592359,-0.590509,0.217311,-1.091895,-1.404649,-1.265719,0.178831,0.689509,-0.938022,-0.972656,0.306869,0.206116,-0.689509,-0.783451,1.306965,-1.152888,-0.427196,-0.901625,-1.733403,0
3,0.240632,0.249898,0.712075,-0.653401,0.177865,-0.167639,0.418306,0.680114,1.511587,1.353458,0.369143,-1.375829,-0.680114,-0.928991,-1.259589,-0.711430,-0.142533,-1.194830,-1.733384,0
4,-0.592359,-0.590509,-1.003112,-0.068741,-0.361748,-1.265719,-0.173125,0.892535,-0.325620,0.423012,-0.307263,0.191639,-0.892535,0.563570,1.306965,-0.213734,-0.199466,0.368928,-1.733374,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,-0.592359,-0.590509,0.875326,-0.848287,0.031008,-1.265719,0.834365,-1.537586,0.899185,0.423012,0.670578,-0.984955,1.537586,0.845396,0.451447,-0.855489,-0.047646,0.662133,1.732372,0
307507,-0.592359,-0.590509,-2.085214,-0.848287,-1.042319,-1.265719,1.164551,1.085707,-1.244224,-0.972656,-0.725959,0.169782,-1.085707,0.310593,3.018001,-0.818594,-0.408219,1.346277,1.732381,0
307508,1.906615,1.930712,0.111636,0.126145,0.198046,-1.265719,-1.144331,-0.245417,-0.938022,0.423012,-1.428203,-0.497002,0.245417,-1.147120,0.451447,0.195379,-0.066623,0.466663,1.732391,0
307509,-0.592359,-0.590509,-0.001203,-0.592499,-0.476315,-0.167639,0.775101,-0.934008,-0.938022,1.353458,1.366859,0.688107,0.934008,-1.124635,-0.404071,-0.568757,0.009287,-1.292565,1.732401,1


#### Training the Neural Network Model

In [49]:
X = app_train_df.drop(columns = ['TARGET'])
y = app_train_df['TARGET']

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

There are two ways to build Keras Models: **Functional and Sequential**.
The **Sequential** API allows you to create models layer by layer

In [54]:
# Creating a sequencial model, defining 3 layers
model = Sequential()
model.add(Dense(12, input_dim=19, activation='relu')) #First hidden layer of the model, 12 Neurons, Num of dimensions independent = 368
model.add(Dense(8, activation='relu')) # Relu to avoid vanishing/exploding problem, 8 neurons
model.add(Dense(1, activation='sigmoid'))# Sigmoid since the output is binary, 1 neuron

# Note that wights and bias are done with Keras default values

In [55]:
#Compiling Model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [56]:
#Training the model
model.fit(X_train, y_train, epochs=1)

7688/7688 [==============================] - 13s 2ms/step - loss: nan - accuracy: 0.9189


In [57]:
#Train set Accuracy
_,accuracy = model.evaluate(X_train, y_train)
print('Train accuracy: %.2f' %(accuracy*100))

7688/7688 [==============================] - 10s 1ms/step - loss: nan - accuracy: 0.9192
Train accuracy: 91.92


In [58]:
#Test set Accuracy
from sklearn.metrics import accuracy_score
y_pred_test = model.predict(X_test)

y_pred_test_df = pd.DataFrame(y_pred_test)
y_pred_test_df.replace([np.inf], np.nan, inplace=True)
y_pred_test_df.dropna(axis=0, inplace=True)


In [59]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score

cf_matrix = confusion_matrix(y_test, y_pred_test)
sns.heatmap(cf_matrix, annot=True)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

## DIFFERENT APPROACH

In [ ]:
# creating a list of 20 one hundred integers 
relu_layers = [100]*20

# Initializing the model
l2_model = keras.models.Sequential()

# Creating 20 - 100 neurons each - hidden layers (relu activation)
for n_hidden in relu_layers:
    l2_model.add(Dense(n_hidden, activation='relu'))
    
# Softmax as the activation function
l2_model.add(Dense(10, activation="softmax", kernel_regularizer=keras.regularizers.l2(0.01)))

# Compiling with a sparse_categorical_crossentropy
l2_model.compile(loss="sparse_categorical_crossentropy", optimizer='adam', metrics=["accuracy"])


In [ ]:
l2_history = l2_model.fit(X_train, y_train, epochs=3, batch_size = 10)

l2_model.evaluate(X_test, y_test)

In [ ]:
pd.DataFrame(l2_history.history).plot(figsize=(10, 7))
plt.grid(True)

plt.show()

In [ ]:
l2_model.evaluate(X_train, y_train)

In [ ]:
l2_model.evaluate(X_test, y_test)